In [7]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import joblib  # For saving models
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE

In [8]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Create output directory if not exists
output_dir = "data/outputs/"
os.makedirs(output_dir, exist_ok=True)

In [9]:
# ---- STEP 1: LOAD DATA ----
file_path = os.path.join(output_dir, "features_insurance_data.csv")
df = pd.read_csv(file_path)

# Define feature columns (X) and target variable (y)
X = df.drop(columns=["churn"])
y = df["churn"]

In [10]:
# ---- STEP 2: TRAIN-TEST SPLIT (80/20) ----
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Training Set: {X_train.shape}, Testing Set: {X_test.shape}")

Training Set: (800, 6), Testing Set: (200, 6)


In [11]:
# ---- STEP 3: HANDLE CLASS IMBALANCE USING SMOTE ----
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Resampled Training Set: {X_train_resampled.shape}")

Resampled Training Set: (1274, 6)


In [12]:
# ---- STEP 4: HYPERPARAMETER TUNING USING CROSS-VALIDATION ----
# Define the hyperparameter grid
param_grid = {
    "n_estimators": [100, 200, 300],  # Number of boosting rounds
    "learning_rate": [0.01, 0.1, 0.2],  # Learning rate
    "max_depth": [3, 6, 9],  # Maximum depth of trees
    "subsample": [0.8, 1.0],  # Fraction of samples used per boosting round
    "colsample_bytree": [0.8, 1.0]  # Fraction of features used per tree
}

# Define the XGBoost model
xgb_model = XGBClassifier(eval_metric="logloss", random_state=42)

# Perform GridSearchCV with 5-fold stratified cross-validation
grid_search = GridSearchCV(xgb_model, param_grid, cv=StratifiedKFold(n_splits=5), scoring="f1", n_jobs=-1)
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model from grid search
best_model = grid_search.best_estimator_
print(f"Best Model Parameters: {grid_search.best_params_}")

Best Model Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 300, 'subsample': 1.0}


In [13]:
# ---- STEP 5: TRAIN THE FINAL MODEL ----
best_model.fit(X_train_resampled, y_train_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [14]:
# ---- STEP 6: MAKE PREDICTIONS ----
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probability scores

In [15]:
# ---- STEP 7: EVALUATE MODEL PERFORMANCE ----
metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1 Score": f1_score(y_test, y_pred),
    "AUC-ROC": roc_auc_score(y_test, y_pred_proba),
}

# Print evaluation metrics
print("\nModel Performance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


Model Performance Metrics:
Accuracy: 0.7100
Precision: 0.1852
Recall: 0.1220
F1 Score: 0.1471
AUC-ROC: 0.4341


In [16]:
# ---- STEP 8: SAVE MODEL PREDICTIONS ----
predictions_df = pd.DataFrame({"Actual": y_test, "Predicted": y_pred, "Predicted_Probability": y_pred_proba})
predictions_file = os.path.join(output_dir, "xgboost_model_predictions.csv")
predictions_df.to_csv(predictions_file, index=False)
print(f"Model Predictions Saved at: {predictions_file}")

Model Predictions Saved at: data/outputs/xgboost_model_predictions.csv


In [17]:
# ---- STEP 9: SAVE THE TRAINED MODEL ----
model_file = os.path.join(output_dir, "xgboost_model.pkl")
joblib.dump(best_model, model_file)
print(f"Trained Model Saved at: {model_file}")

print("\nXGBoost Model Training Completed Successfully!")

Trained Model Saved at: data/outputs/xgboost_model.pkl

XGBoost Model Training Completed Successfully!
